# RadarPillar: Google Colab Training Template

Bu notebook, **RadarPillar** projesini Google Colab üzerinde çalıştırmak için hazırlanmıştır.

## 1. Hazırlık ve GPU Kontrolü

In [ ]:
# GPU durumunu kontrol et (T4, V100 veya A100 olması önerilir)
!nvidia-smi

import torch
print(f"PyTorch Versiyonu: {torch.__version__}")
print(f"CUDA Erişilebilir: {torch.cuda.is_available()}")

## 2. Google Drive Bağlantısı

Veri setlerini ve çıktıları kalıcı olarak saklamak için Drive'ı bağlayın.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 3. Kurulum (Estatik Bağımlılıklar)

Proje kodunu klonlayın ve gerekli kütüphaneleri kurun.

In [ ]:
%cd /content

# Repo URL'sini kendi reponuzla değiştirin
!git clone https://github.com/YOUR_USERNAME/RadarPillar.git
%cd RadarPillar

# Temel kütüphaneler
!pip install numpy easydict pyyaml scikit-image tqdm tensorboardX numba wandb

# spconv-cuXXX (Colab CUDA versiyonuna göre seçilmelidir. Genelde cu121 veya cu118)
import os
cuda_version = os.popen('nvcc --version').read()
if '12.2' in cuda_version or '12.1' in cuda_version:
    !pip install spconv-cu121
else:
    !pip install spconv-cu118

# OpenPCDet (pcdet) kurulumu
!python setup.py develop

## 4. Veri Seti Yapılandırması

Drive'daki VoD veri setini projenin `data` klasörüne sembolik link ile bağlayın.

In [ ]:
import os

# DRIVE_VOD_PATH değerini Drive'daki gerçek yolunuzla değiştirin
DRIVE_VOD_PATH = "/content/drive/MyDrive/VoD"
LOCAL_DATA_PATH = "/content/RadarPillar/data/VoD"

if os.path.exists(DRIVE_VOD_PATH):
    os.makedirs(os.path.dirname(LOCAL_DATA_PATH), exist_ok=True)
    if not os.path.exists(LOCAL_DATA_PATH):
        os.symlink(DRIVE_VOD_PATH, LOCAL_DATA_PATH)
        print("✓ VoD yolu başarıyla bağlandı.")
    else:
        print("! Yerel veri yolu zaten mevcut.")
else:
    print("✘ HATA: Drive üzerindeki VoD yolu bulunamadı!")

## 5. Info Dosyalarının Oluşturulması

Eğitime başlamadan önce veri seti infolarını üretmeniz gerekir.

In [ ]:
%cd /content/RadarPillar

# VoD Radar-only infoları
!python -m pcdet.datasets.vod.vod_dataset create_vod_infos \
    tools/cfgs/dataset_configs/vod_dataset_radar.yaml

## 6. Eğitim

WandB kullanmak istiyorsanız hücreyi çalıştırmadan önce `wandb login` yapmanız önerilir.

In [ ]:
# !wandb login  # İlk seferde API KEY girmeniz gerekebilir

!python tools/train.py \
    --cfg_file tools/cfgs/vod_models/vod_radarpillar.yaml \
    --batch_size 4 \
    --epochs 80 \
    --use_wandb

## 7. Çıktıları Drive'a Yedekleme

Colab oturumu kapandığında dosyalar silinir. Checkpoint'leri Drive'a kopyalamayı unutmayın.

In [ ]:
import shutil

SOURCE = "/content/RadarPillar/output"
DESTINATION = "/content/drive/MyDrive/RadarPillar_Backup"

if os.path.exists(SOURCE):
    shutil.copytree(SOURCE, DESTINATION, dirs_exist_ok=True)
    print(f"✓ Tüm çıktılar Drive'a kopyalandı: {DESTINATION}")